# Mastodon API operations

API operations defined in `mastodon_sim.mastodon_ops`.

In [ ]:
%load_ext autoreload
%autoreload 2

## Check environment

Get environment variables with `.env` file (in project root directory)

In [ ]:
from mastodon_sim.mastodon_ops import check_env

check_env()

## Get Mastodon client

Create Mastodon client object

In [ ]:
from mastodon_sim.mastodon_ops import get_client

mastodon = get_client()

## Login user

Get access token for `user0001`.

In [ ]:
from mastodon_sim.mastodon_ops import login

access_token = login("user0001")
print(f"access_token: {access_token}")

## Update name and bio

Update `user0001`'s display name and bio.

In [ ]:
from mastodon_sim.mastodon_ops import update_bio

display_name = "Bob"
bio = "Hello, I am user0001!"

update_bio(login_user="user0001", display_name=display_name, bio=bio)

## Read name and bio

Have one user read another user's bio. Defaults to "Not provided" if the user has not set either.

In [ ]:
from mastodon_sim.mastodon_ops import read_bio

display_name, bio = read_bio(login_user="user0002", target_user="user0001")

display_name, bio = read_bio(login_user="user0001", target_user="user0002")

## Follow user

Make `user0001` follow `user0002` and vice versa.

In [ ]:
from mastodon_sim.mastodon_ops import follow

follow(login_user="user0001", follow_user="user0002")

follow(login_user="user0002", follow_user="user0001")

## Unfollow user

Make `user0002` unfollow `user0001`.

In [ ]:
from mastodon_sim.mastodon_ops import unfollow

unfollow(login_user="user0002", unfollow_user="user0001")

## Toot (post a status)

Make `user0001` and `user0002` toot a `status`.

Note that "toot" is analogous to "tweet".

In [ ]:
from mastodon_sim.mastodon_ops import toot

toot(login_user="user0001", status="Hello World, from user0001!")

toot(login_user="user0002", status="Hello World, from user0002!")

## Post a status with additional settings

Note that `toot` is a synonym for `post_status`, but only takes the status text as input.

In [ ]:
from mastodon_sim.mastodon_ops import post_status

post_status(
    login_user="user0002",
    status="Hello World, from user0002! This is an unlisted toot with a content warning.",
    visibility="unlisted",
    sensitive=True,
    spoiler_text="Content warning",
)

## Send a DM

Send a DM from `user0001` to `user0002`.

In [ ]:
post_status(
    login_user="user0001",
    status="@user0002 Hello, this is a direct message!",
    visibility="direct",
)

###

## Post with media attachments

In [ ]:
image_file = "../infrastructure/mastodon-on-aws/architecture.png"

post_status(
    login_user="user0001",
    status="Check out this image!",
    media_files=[image_file],
)

## Create a poll

In [ ]:
post_status(
    login_user="user0001",
    status="What's your favorite color?",
    poll_options=["Red", "Blue", "Green"],
    poll_expires_in=86400,
)

## Get the public timeline

Get the entire pubic timeline in reverse chronological order, up to `limit`.

In [ ]:
from mastodon_sim.mastodon_ops import get_public_timeline, print_timeline

timeline = get_public_timeline(limit=None)
print_timeline(timeline)

## Get own timeline

Retrieve `user0001`'s timeline. Fetches the logged-in user's home timeline (i.e., followed users and self). Offers filtering options:
- `all`: Default, shows all posts
- `self`: Shows only the user's own posts
- `others`: Shows only posts from followed users

In [ ]:
from mastodon_sim.mastodon_ops import get_own_timeline

# Notice the retrieved count is different for each of these
timeline = get_own_timeline("user0001", filter_type="all")
timeline = get_own_timeline("user0001", filter_type="self")
timeline = get_own_timeline("user0001", filter_type="others")

# Print the timeline (in this case, all others following, but not self)
print_timeline(timeline)

## Read a user's timeline (non-global)

From the perspective of `user0002`, read `user0001`'s timeline.

**Accessible Information**:
  - `user0002` can read `user0001`'s public and unlisted posts, as well as follower-only posts if `user0002` is a follower.
  - Details available include post ID, creation date, username, display name, content, URL, favourites count, and reblogs count.

**Restricted Information**:
  - `user0002` cannot access private posts or direct messages.

**Impact of Being Blocked**:
  - If `user0001` has blocked `user0002`, `user0002` cannot retrieve any posts from `user0001`'s timeline.
  - The function will return an empty list due to the block enforced by the Mastodon API.

In [ ]:
from mastodon_sim.mastodon_ops import get_user_timeline

timeline = get_user_timeline(login_user="user0002", target_user="user0001", limit=None)
print_timeline(timeline)

## Block / unblock a user

Make `user0001` block or unblock `user0002`.

In [ ]:
from mastodon_sim.mastodon_ops import block_user, unblock_user

# user001 blocks user002
block_user(login_user="user0001", target_user="user0002")

# user002 tries to read user001's timeline, but retrieves zero toots
timeline = get_user_timeline(login_user="user0002", target_user="user0001")

# user001 unblocks user002
unblock_user(login_user="user0001", target_user="user0002")

# user002 tries to read user001's timeline again, and retrieves the toots
timeline = get_user_timeline(login_user="user0002", target_user="user0001")

## Mute account (user)

Make `user0001` mute and unmute `user0002`.

In [ ]:
from mastodon_sim.mastodon_ops import mute_account, unmute_account

mute_account(
    login_user="user0001",
    mute_user="user0002",
    notifications=False,
    duration=None,
)

unmute_account(
    login_user="user0001",
    unmute_user="user0002",
)

## Like (favorite) a toot

Make `user0002` like `user0001`'s toot with ID `toot_id`.

In [ ]:
from mastodon_sim.mastodon_ops import like_toot

timeline = get_user_timeline(login_user="user0002", target_user="user0001")

# Most recent toot is index 0 (reverse chronological order)
most_recent_toot_id = timeline[0]["id"]

like_toot(login_user="user0002", target_user="user0001", toot_id=most_recent_toot_id)

## Boost (reblog) a toot

Make `user0002` boost `user0001`'s toot with ID `toot_id`.

In [ ]:
from mastodon_sim.mastodon_ops import boost_toot

timeline = get_user_timeline(login_user="user0002", target_user="user0001")
most_recent_toot_id = timeline[0]["id"]

boost_toot(login_user="user0002", target_user="user0001", toot_id=most_recent_toot_id)

## Post a reply

Make `user0002` reply to `user0001`'s post (a poll).

In [ ]:
timeline = get_user_timeline(login_user="user0002", target_user="user0001")
print(f"Last toot: {timeline[0]['content']}")
most_recent_toot_id = timeline[0]["id"]

post_status(
    login_user="user0002",
    status="This is a great poll!",
    in_reply_to_id=most_recent_toot_id,
)

## Read notifications

Read all notifications for `user0001` and clear them.

In [ ]:
from mastodon_sim.mastodon_ops import print_notifications, read_notifications

notifications = read_notifications(login_user="user0001", clear=True, limit=None)
print_notifications(notifications)

## Display user follow graph

In [ ]:
from mastodon_sim.mastodon_ops import follow
from mastodon_sim.mastodon_utils import create_user_graph

# Set up some follows
follow(login_user="user0001", follow_user="user0002")
follow(login_user="user0002", follow_user="user0001")
follow(login_user="user0003", follow_user="user0001")
follow(login_user="user0004", follow_user="user0001")
follow(login_user="user0005", follow_user="user0001")

create_user_graph()  # Note: interactive graph — not visible in GitHub

## Delete post(s)

Make `user0002` delete their last post (the reply to `user0001`'s poll).

In [ ]:
# from mastodon_sim.mastodon_ops import delete_posts

# # Check user0002's number of toots
# timeline = get_own_timeline("user0002")

# delete_posts(
#     login_user="user0002",
#     recent_count=1,
# )

# # Check user0002's number of toots again, should be 1 less
# timeline = get_own_timeline("user0002")

Delete a post by post ID.

In [ ]:
# # Get user0002's first toot (and print the number of toots)
# timeline = get_own_timeline("user0002")
# first_toot_id = timeline[0]["id"]

# delete_posts(
#     login_user="user0002",
#     post_ids=[first_toot_id],
# )

# # Check user0002's number of toots again, should be 1 less
# timeline = get_own_timeline("user0002")

## Delete all posts

Delete all posts for a list of users.

In [ ]:
# for user in ["user0001", "user0002"]:
#     delete_posts(login_user=user, delete_all=True, skip_confirm=True)

## Reset all users

Delete user's posts, likes, boosts, and reset display name and bio.

In [ ]:
from mastodon_sim.mastodon_ops import reset_users
from mastodon_sim.mastodon_utils import get_users_from_env

# Get all users from .env file
users = get_users_from_env()

# Reset all users
reset_users(users, skip_confirm=True, parallel=True)

# Check that the public timeline is now empty
assert not len(get_public_timeline(limit=None))

And that's it so far!

## [TODO] 

### Operations to add:

- favourites
- bookmarks
- timeline_hashtag
- conversations
- trending_tags
- status_update
- admin_account_moderate

https://mastodonpy.readthedocs.io/en/stable/index.html